## Web Service Classifcation using BERT for top 10,20,30,40,50 Web Service Categories
## Web Service Classification using BERT (RoBERTa)

## 1.  Load Python Modules

In [13]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
# Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from transformers import (
    RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
)
import warnings
import torch
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
%matplotlib inline

sns.set(style="whitegrid")
os.makedirs("charts", exist_ok=True)
os.makedirs("results", exist_ok=True)
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Ensure deterministic behavior (optional: slower)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

cuda


## 2. Generate Report in CSV

In [14]:
def generate_report_csv(report_dict, confusion_mat, class_labels, class_names, output_file="BERT-Report.csv"):
    df = pd.DataFrame(report_dict).transpose()

    # Filter only the class rows (not avg/macro)
    filtered_labels = [str(class_names[i]) for i in class_labels]
    df = df.loc[filtered_labels].copy()

    # Compute per-class accuracy
    row_sums = confusion_mat.sum(axis=1)
    diagonal = np.diag(confusion_mat)
    per_class_accuracy = (diagonal / row_sums).round(2)

    df['accuracy'] = per_class_accuracy
    if 'support' in df.columns:
        df = df.drop(columns=['support'])

    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Category'}, inplace=True)

    output_path = f"results/{output_file}"
    df.to_csv(output_path, index=False)
    print(f"Classification report saved to: {output_path}")

## 3. Model -Helper-functions

In [15]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average='weighted', zero_division=0),
        "recall": recall_score(labels, preds, average='weighted', zero_division=0),
        "f1": f1_score(labels, preds, average='weighted', zero_division=0)
    }

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

def plot_confusion_and_report(y_true, y_pred, class_names, n):
    cm = confusion_matrix(y_true, y_pred)
    figsize = (18, 18) if n >= 40 else (8, 8)
    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix - Top {n} Categories")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"charts/BERT_confusion_matrix_top_{n}.png")
    plt.show()

    print(f"classification_report for {n} Web Serivce Categories")
    # Convert class_names to list of strings
    class_names_str = [str(name) for name in class_names]
    
    # Get sorted list of unique label integers
    labels = sorted(np.unique(y_true))

    # classification_report for print
    print(classification_report(y_true, y_pred, target_names=class_names_str, zero_division=0))

    # classification_report for dictionary
    report_dict = classification_report(y_true, y_pred, target_names=class_names_str, output_dict=True, zero_division=0)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Save to CSV
    generate_report_csv(report_dict, cm, class_labels=labels, class_names=class_names_str, output_file=f"BERT-Report-Top-{n}.csv")

    return report_dict

def plot_metrics(bert_results):
    categories = list(bert_results.keys())
    accuracies = [bert_results[n]['accuracy'] for n in categories]
    precisions = [bert_results[n]['precision'] for n in categories]
    recalls = [bert_results[n]['recall'] for n in categories]
    f1_scores = [bert_results[n]['f1_score'] for n in categories]

    def plot_metric(x, y, ylabel, title, color='blue'):
        plt.figure(figsize=(8, 5))
        plt.plot(x, y, marker='o', linestyle='-', color=color)
        plt.title(title)
        plt.xlabel('Number of Web Serivce Categories')
        plt.ylabel(ylabel)
        plt.grid(True)
        plt.xticks(x)
        plt.tight_layout()
        plt.savefig(f"charts/{title}.png")
        plt.show()

    plot_metric(categories, accuracies, 'Accuracy', 'BERT Model Performace Accuracy', color='green')
    plot_metric(categories, precisions, 'Precision', 'BERT Model Performace Precision', color='orange')
    plot_metric(categories, recalls, 'Recall', 'BERT Model Performace Recall', color='purple')
    plot_metric(categories, f1_scores, 'F1 Score', 'BERT Model Performace F1 Score', color='red')


## 4. Pre Processing Text

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
# Initialize tools
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]

    return " ".join(lemmatized_tokens)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91956\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91956\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91956\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\91956\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\91956\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## 5. Train and Evaluate

In [17]:
def train_and_evaluate(n):
    df = pd.read_csv(f"../data-preprocessing/Pre_Processed_Top_{n}_Web_Services_Categories.csv").dropna()
    df["Service Description"] = df["Service Description"].apply(clean_text)
    
    df = df[['Service Description', 'Service Classification']].rename(columns={'Service Description': 'text', 'Service Classification': 'label'})

    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])
    num_labels = df['label'].nunique()
    class_names = label_encoder.classes_

    dataset = Dataset.from_pandas(df.reset_index(drop=True))
    dataset = dataset.train_test_split(test_size=0.2, seed=42)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns(["text"])

    model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)
    training_args = TrainingArguments(
        output_dir=f"./results_top_{n}",
        num_train_epochs=3,
        eval_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir=f"./logs_top_{n}",
        logging_strategy="steps",
        logging_steps=100,
        seed=42,  
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    preds = trainer.predict(tokenized_dataset['test'])
    y_true = preds.label_ids
    y_pred = np.argmax(preds.predictions, axis=1)

    metrics = compute_metrics((preds.predictions, y_true))

    report = plot_confusion_and_report(y_true, y_pred, class_names, n)
    

    return {
        "accuracy": metrics["accuracy"],
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "f1_score": metrics["f1"]
    }


## 6. Run the Model

In [ ]:
# Run and collect results
bert_results = {}
for n in [10, 20, 30, 40, 50]:
    print(f"\n Processing Top {n} Web Service Categories")
    bert_results[n] = train_and_evaluate(n)

# Save results
with open("results/bert_roberta_results.pkl", "wb") as f:
    pickle.dump(bert_results, f)


 Processing Top 10 Web Service Categories


Map:   0%|          | 0/3761 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


## 7. Performance and Plots

In [ ]:
# Plot metrics
plot_metrics(bert_results)